This will take a user input and run it through the model that was saved in _05_mlp_analysis

In [1]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from torch.utils.data import SubsetRandomSampler
import sys
import os

In [3]:
#add the parent directory to the current working directory so that we can access MLP class
repository_root_directory = os.path.dirname(os.getcwd())
rrd = "repository_root_directory:\t"
print(rrd, repository_root_directory)

if repository_root_directory not in sys.path:
    sys.path.append(repository_root_directory)
    print(rrd, "added to path")
else:  
    print(rrd, "already in path")

repository_root_directory:	 c:\Users\cengl\Documents\CSC 461\code\csc_461_fp
repository_root_directory:	 already in path


In [4]:
#import the class
from mlp import MLP

#check to see if cuda is avalable
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("cuda status: ", device)


#create a configuration dictionary. input size is the number of features in each sample
config = {
    'input_size': 30,
    'output_size': 10,
    'hidden_layers': [256,64],
    'batch_size': 64,
    'n_epochs': 20,
    'learning_rate': 0.001
}

model = MLP(config['input_size'], config['output_size'], config['hidden_layers']).to(device)
#define criterion and an optimizer
#criterion = nn.CrossEntropyLoss()
#optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'])

# Load the saved state dictionary
savedModel = torch.load('model.pth')
model.load_state_dict(torch.load('model.pth'))
#model.load_state_dict(savedModel['model_state_dict'])
#optimizer.load_state_dict(savedModel['optimizer_state_dict'])

#startEpoch = savedModel['epoch']
#lossValue = savedModel['loss']
# Set the model to evaluation mode
model.eval()


cuda status:  cuda


C:\Users\cengl\AppData\Local\Temp\ipykernel_1332\186475717.py:25: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  savedModel = torch.load('model.pth')
C:\Users\cengl\AppData\L

MLP(
  (net): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=30, out_features=256, bias=True)
    (2): ReLU()
    (3): Linear(in_features=256, out_features=64, bias=True)
    (4): ReLU()
    (5): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [ ]:
#this will load a song and output the genre
from data_preprocessor import DataPreprocessor
from data_preprocessor_parallel_proc import DataPreprocessorParallelProc
from pathlib import Path
import librosa

sample_rate = 22050                     # default sample rate of the dataset
preprocessor = DataPreprocessor()
audioProcessor = DataPreprocessorParallelProc()

#get the path to the file
relative_path = Path('..') / 'song1.mp3'

# Get the absolute path
file_path = os.path.abspath(relative_path)

print(f"The absolute path is: {file_path}")

print('Current working directory:', os.getcwd())

# Preprocess the audio
try:
            # Load the audio file with the correct sample rate
            y, sr = librosa.load(file_path, sr=sample_rate) if sample_rate != 22050 else librosa.load(file_path, sr=None)

            # Normalize the audio
            y = librosa.util.normalize(y)

            # Trim leading and trailing silence
            y, _ = librosa.effects.trim(y)

            
except Exception as e:
    print(f"Error preprocessing file {file_path}: {e}")
print("y is: ", y, "sr is: ", sr)


#extract the features
from features_extractor import FeaturesExtractor

feature_extractor = FeaturesExtractor()
data = feature_extractor.extract_features_from_file(y,sr)
print("Data is: ",data)

#convert the data to a tensor
features = torch.tensor(data, dtype=torch.float32).unsqueeze(0) #make the data a tensor
features = features.to(device) #make sure that it is using the same device. 

#run the data through the model and get the predicted genre
with torch.no_grad():  # No gradients needed for inference

    output = model(features)  # Forward pass
    predicted_class = torch.argmax(output, dim=1).item() 


genre_mapping = {0: 'blues', 1: 'Classical', 2: 'Country', 3: 'disco',4: 'hiphop', 5: 'jazz', 6: 'metal', 7: 'pop',8: 'reggae', 9: 'rock'}  # Adjust based on your data
predicted_genre = genre_mapping[predicted_class]
print(f"Predicted Genre: {predicted_genre}")

print("output is: ", output)
print("predicted class is: ", predicted_class)
print("that class is: ", predicted_genre)


The absolute path is: c:\Users\cengl\Documents\CSC 461\code\csc_461_fp\song1.mp3
Current working directory: c:\Users\cengl\Documents\CSC 461\code\csc_461_fp\notebooks
y is:  [-9.8376831e-06 -2.1273676e-05 -3.0678435e-05 ... -3.1345972e-04
 -3.0257838e-04 -2.9095120e-04] sr is:  44100
Data is:  [ 9.71570686e-02  4.05669370e-02  1.14922460e+03  1.04892204e+03
  2.09968816e+03 -2.85636871e+02  2.02346115e+02 -2.25924473e+01
 -3.11582118e-01  1.41619005e+01 -6.03861284e+00 -1.48845530e+00
 -7.81741142e+00 -6.15064573e+00 -7.04260397e+00 -1.34700441e+01
 -6.61526585e+00 -4.52209520e+00  3.15452009e-01  2.97938824e-01
  2.85389543e-01  4.01578158e-01  4.38602120e-01  3.42290670e-01
  2.82668203e-01  3.34119618e-01  4.15963143e-01  3.53892475e-01
  3.58989269e-01  3.21403474e-01]
Predicted Genre: Classical
output is:  tensor([[17.8418, 21.7945, 14.1225, 12.7982, 11.1918, 14.0066, 12.5498,  7.0805,
         15.8143,  8.6845]], device='cuda:0')
predicted class is:  1
that class is:  Classical
